In [1]:
import time 
start=time.time()

In [2]:
#importing libraries

import gensim
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import csv
import re
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\NR\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
#Loading pre=trained word2vec model

from gensim.models.keyedvectors import KeyedVectors

# You need to dowload google pre-trained model using below link
# https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit
#Change the path according to your directory

model_path = 'D:\GoogleNews_vectors_negative300\GoogleNews_vectors_negative300.bin'   
w2v_model = KeyedVectors.load_word2vec_format(model_path, binary=True)

In [4]:
# Importing Source Rules or OCD Rulebook CSV file

with open ('Target Rules or EA rulebook.csv') as file_object:
    for OCDline in file_object:
        print(OCDline)

2.1.3 Controls – Operations under degraded conditions

2.1.1.4 - Written operating procedures or spoken instructions when the signalling system cannot be used to preserve the effectiveness of the space interval

2.5.2.1 trains not permitted to start journey unless there is sufficient braking capacity

2.3.2 Controls – Level Crossings

2.6.1.4 Route availability system provides consistent and simple method to assess compatibility of train weight with underlying bridges

2.2.1.1 Safety is compromised if a train proceeds without a movement autority

2.8.6 Control communication between train drivers and signallers



In [5]:
#Sentences tokenization of Source Rules or OCD Rulebook CSV file

target_rules = []
with open ('Target Rules or EA rulebook.csv') as file_object:
    for trainline in file_object:
        tokens_train = sent_tokenize(trainline)
        target_rules.extend(tokens_train)

    
# Searching for text that includes any digits or decimal number. "\d" is looking for digits in tokens

withdigits_target_rules=[s for s in target_rules if re.findall("^\d",s)]
print(withdigits_target_rules)

['2.1.3 Controls – Operations under degraded conditions', '2.1.1.4 - Written operating procedures or spoken instructions when the signalling system cannot be used to preserve the effectiveness of the space interval', '2.5.2.1 trains not permitted to start journey unless there is sufficient braking capacity', '2.3.2 Controls – Level Crossings', '2.6.1.4 Route availability system provides consistent and simple method to assess compatibility of train weight with underlying bridges', '2.2.1.1 Safety is compromised if a train proceeds without a movement autority', '2.8.6 Control communication between train drivers and signallers']


In [6]:
# Importing Target Rules or EA Rulebook CSV file

with open ('Source rules or OCD rulebook.csv') as file_object:
    for EAline in file_object:
        print(EAline)

2.1.3 Controls – Operations under degraded conditions

2.1.1.4 Operating procedures using written or spoken instructions are applied when the signalling system cannot be used to preserve the effectiveness of the space interval:

2.5.2.1 Trains must not be permitted to start a journey unless there is sufficient braking capacity available for the complete train.  

2.3.2 Controls – Level Crossings

2.6.1.4 The Route Availability (RA) system provides a consistent and simple method for assessing the compatibility of the weight of rail vehicles with the capacity of underline bridges (other than bridges defined as ‘long span’).

2.2.1.1 The safety benefits of a system for maintaining space intervals between trains (see section 2.1 of this operational concept document) are compromised if a train proceeds without an authority for its movement.

2.8.6 Controls – railway workforce (Communication between train drivers and signallers)



In [7]:
#Sentences tokenization of Target Rules or EA Rulebook CSV file

source_rules = []
with open ('Source rules or OCD rulebook.csv') as file_object:
    for trainline in file_object:
        tokens_train = sent_tokenize(trainline)
        source_rules.extend(tokens_train)

        
# Searching for text that includes any digits or decimal number. "\d" is looking for digits in tokens


withdigits_source_rules=[s for s in source_rules if re.findall("^\d",s)]
print(withdigits_source_rules)

['2.1.3 Controls – Operations under degraded conditions', '2.1.1.4 Operating procedures using written or spoken instructions are applied when the signalling system cannot be used to preserve the effectiveness of the space interval:', '2.5.2.1 Trains must not be permitted to start a journey unless there is sufficient braking capacity available for the complete train.', '2.3.2 Controls – Level Crossings', '2.6.1.4 The Route Availability (RA) system provides a consistent and simple method for assessing the compatibility of the weight of rail vehicles with the capacity of underline bridges (other than bridges defined as ‘long span’).', '2.2.1.1 The safety benefits of a system for maintaining space intervals between trains (see section 2.1 of this operational concept document) are compromised if a train proceeds without an authority for its movement.', '2.8.6 Controls – railway workforce (Communication between train drivers and signallers)']


In [8]:
#Setting Parameters for model

class DocSim(object):
    def __init__(self, w2v_model , stopwords=[]):
        self.w2v_model = w2v_model
        self.stopwords = stopwords

    def vectorize(self, doc):
        """Identify the vector values for each word in the given document"""
        doc = doc.lower()
        words = [w for w in doc.split(" ") if w not in self.stopwords]
        word_vecs = []
        for word in words:
            try:
                vec = self.w2v_model[word]
                word_vecs.append(vec)
            except KeyError:
                # Ignore, if the word doesn't exist in the vocabulary
                pass

        # Assuming that document vector is the mean of all the word vectors

        vector = np.mean(word_vecs, axis=0)
        return vector


    def _cosine_sim(self, vecA, vecB):
        """Find the cosine similarity distance between two vectors."""
        csim=(( np.dot(vecA,vecB) )**2)/((np.linalg.norm(vecA)*np.linalg.norm(vecB))**2)     
        if np.isnan(np.sum(csim)):                   #Test element-wise for NaN and return result as a boolean array.
            return 0
        return csim

    def calculate_similarity(self, withdigits_source_rules, withdigits_target_rules=[], threshold=0.4):
        """Calculates & returns similarity scores between given source rules & all
        the target rules"""
        if isinstance(withdigits_target_rules, str):
            withdigits_target_rules = [withdigits_target_rules]


        source_vec = self.vectorize(withdigits_source_rules)
        results = []
        
        for rule in withdigits_target_rules:
            target_vec = self.vectorize(rule)
            sim_score = self._cosine_sim(source_vec, target_vec)
            if sim_score > threshold:
                results.append({
                    'Cosine Sim Score':sim_score,
                    'Target Rule':rule
                })

                
            # Sort results by score in desc order
            results.sort(key=lambda k : k['Cosine Sim Score'] , reverse=True)

        return results


In [9]:
ds = DocSim(w2v_model)

In [10]:
#Calculate the similarity score between a source rule & a target rule.


if isinstance(withdigits_source_rules, str):
    withdigits_source_rules = [withdigits_source_rules]
   

# This will return one target rules text with a similarity score

for rule in withdigits_source_rules:
    sim_scores= ds.calculate_similarity(rule, withdigits_target_rules)

    
    
    # Source Rules are OCD Rules
    # Target Rules are EA Rules
    
    # Printing the output in text file
    
    print("Source rule: {} \n\nSimilarity with Target Rule is \n\n {}\n".format(rule, sim_scores) , file=open("output.txt", "a"))
    print("\n")
    
    
    # Printing output in Jupyter
    
    print("Source rule: {} \n\nSimilarity with Target Rule is \n\n {}\n".format(rule, sim_scores) )
    print("\n")



Source rule: 2.1.3 Controls – Operations under degraded conditions 

Similarity with Target Rule is 

 [{'Cosine Sim Score': 1.0, 'Target Rule': '2.1.3 Controls – Operations under degraded conditions'}]





Source rule: 2.1.1.4 Operating procedures using written or spoken instructions are applied when the signalling system cannot be used to preserve the effectiveness of the space interval: 

Similarity with Target Rule is 

 [{'Cosine Sim Score': 0.9168542737999658, 'Target Rule': '2.1.1.4 - Written operating procedures or spoken instructions when the signalling system cannot be used to preserve the effectiveness of the space interval'}]





Source rule: 2.5.2.1 Trains must not be permitted to start a journey unless there is sufficient braking capacity available for the complete train. 

Similarity with Target Rule is 

 [{'Cosine Sim Score': 0.8824563091599441, 'Target Rule': '2.5.2.1 trains not permitted to start journey unless there is sufficient braking capacity'}, {'Cosine Sim

In [11]:
done=time.time()
elapsed=done-start
print(elapsed)

112.31474256515503
